In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [3]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
from orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [4]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kang/Data/TSLA_2015-01-02_34200000_57600000_orderbook_10.csv'

In [5]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/project-RL4ABM/data_tqap/TSLA_2015-01-01_2015-01-31_10/TSLA_2015-01-02_34200000_57600000_message_10.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/project-RL4ABM/data_tqap/TSLA_2015-01-01_2015-01-31_10/TSLA_2015-01-02_34200000_57600000_orderbook_10.csv", header=None)

In [6]:
data=jnp.array(df_ob.iloc[0,:]).reshape(20,2)
newarr=jnp.zeros((20,7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:40:2,1].set(-1).at[1:40:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders_simple)
print(ordersJNP.shape)

(212966, 7)


In [7]:
orderbook=JaxOrderBook()
trades_init=orderbook.process_orders_array(initOB)
trades_msgs=orderbook.process_orders_array(ordersJNP[0:35])

In [8]:
orderbook.orderbook_array

Array([[[[ 1.5070000e+03,  2.2291000e+06,  0.0000000e+00,
          -9.9999900e+05,  3.4200000e+04],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000000e+00],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
          -1.0000000e+00, -1.0000